In [3]:
!pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/b9/88/f6b026a424d66d185534cb356fecaa63c96540227c306b2d96b61385f8d1/tensorflow-2.0.0-cp35-cp35m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/5e/f8/82a8a6ed446b58aa718b2744b265983783a2c84098a73db6d0b78a573e25/numpy-1.17.3-cp35-cp35m-manylinux1_x86_64.whl
     |████████████████████████████████| 378kB 751kB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 12.2MB/s eta 0:00:01
Processing /home/guiz/.cache/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833/opt_einsum-3.1.0-cp35-none-any.whl
Processing /home/guiz/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd/gast-0.2.2-cp35-none-any.whl
     |████████████████████████████████| 450kB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 178kB/s eta 0:00:01
     |████████████████████████████████| 81kB 1.5MB/s eta 0:00:011
     |████████████████████████████████| 133kB 7.7MB/s eta 0:00

     |████████████████████████████████| 61kB 139kB/s  eta 0:00:01
     |████████████████████████████████| 81kB 45kB/s  eta 0:00:011
     |████████████████████████████████| 61kB 87kB/s s eta 0:00:01
     |████████████████████████████████| 143kB 11.7MB/s eta 0:00:01
     |████████████████████████████████| 133kB 7.2MB/s eta 0:00:01
     |████████████████████████████████| 163kB 9.5MB/s eta 0:00:01
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.24.0 which is incompatible.
  Found existing installation: numpy 1.16.2
    Uninstalling numpy-1.16.2:
      Successfully uninstalled numpy-1.16.2
  Found existing installation: gast 0.3.2
    Uninstalling gast-0.3.2:
      Successfully uninstalled gast-0.3.2
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Succ

In [7]:
# Convolutional Neural Network

# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [8]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [4]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000)

Using TensorFlow backend.


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


NameError: name 'classifier' is not defined

In [10]:
# Part 3 - Making new predictions

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.